# Constraint Satisfaction Homework

CSP domain, variables, values and constraints for this problem.

    - Variables = Teams
    - Variable Assignment = Group
    - Domain = Pot
    - Constraint = no two teams from same pot, and no two teams from same confederate, unless UEFA are allowed in same group. If from UEFA then two teams are allowed.

## Code in hw2cs561s2018.py file

### Read in file

In [98]:
#import sys
#filePath = sys.argv[1]
with open('tests/input1.txt', 'r') as openFile:
  fileData = openFile.readlines()
openFile.close()
cspInfo = [line.strip() for line in fileData]



### Assign variables from values provided in value

In [99]:
import collections
dNumOfGroups = int(cspInfo[0])
dNumOfPots = int(cspInfo[1])
lsVariables = []
dctVarInfo = collections.defaultdict(list)
dctDomains = collections.defaultdict(list)
count = 2
intPotNum = 1
for pot in range(dNumOfPots):
    splitLine = cspInfo[count].split(',')
    for country in splitLine:
        lsVariables.append(country)
        dctVarInfo[country].append(intPotNum)

        for num in range(1, dNumOfGroups+1):
            dctDomains[country].append(num)

    count = count + 1
    intPotNum = intPotNum + 1
for confederation in range(6):
    #parse string
    splitLine = cspInfo[count].split(':')
    strConfederation = splitLine.pop(0)
    splitLine = str(splitLine)
    splitLine = splitLine.split(',')
    count = count + 1
    if 'None' not in ''.join(splitLine):
        for word in splitLine:
            word = word.strip('[')
            word = word.strip(']')
            word = word.strip("'")
            dctVarInfo[word].append(strConfederation)

In [100]:
# Create neighbors variable and assign neighbors related to values that 
# can be assigned. For example: If variable country is in pot 1; then its
# neighbors can be 2, 3, and/or 4.
dctConfNeighbors = collections.defaultdict(list)
dctPotNeighbors = collections.defaultdict(list)

for country in dctVarInfo.keys():
    if dctVarInfo[country][0] == 1:
        neigh = [2,3,4]
        dctPotNeighbors[country].append(neigh)
    elif dctVarInfo[country][0] == 2:
        neigh = [1,3,4]
        dctPotNeighbors[country].append(neigh)
    elif dctVarInfo[country][0] == 3:
        neigh = [1,2,4]
        dctPotNeighbors[country].append(neigh)
    else:
        neigh = [1,2,3]
        dctPotNeighbors[country].append(neigh)
    if dctVarInfo[country][1] == 'AFC':
        neigh = ['CAF', 'OFC', 'CONCACAF', 'CONMEBOL', 'UEFA']
        dctConfNeighbors[country].append(neigh)
    elif dctVarInfo[country][1] == 'CAF':
        neigh = ['AFC', 'OFC', 'CONCACAF', 'CONMEBOL', 'UEFA']
        dctConfNeighbors[country].append(neigh)
    elif dctVarInfo[country][1] == 'OFC':
        neigh = ['AFC', 'CAF', 'CONCACAF', 'CONMEBOL', 'UEFA']
        dctConfNeighbors[country].append(neigh)
    elif dctVarInfo[country][1] == 'CONCACAF':
        neigh = ['AFC', 'CAF', 'OFC', 'CONMEBOL', 'UEFA']
        dctConfNeighbors[country].append(neigh)
    elif dctVarInfo[country][1] == 'CONMEBOL':
        neigh = ['AFC', 'CAF', 'OFC', 'CONCACAF', 'UEFA']
        dctConfNeighbors[country].append(neigh)
    else:
        neigh = ['AFC', 'CAF', 'OFC', 'CONCACAF', 'CONMEBOL']
        dctConfNeighbors[country].append(neigh)

### Send variables to CSP file to be solved and returned.

In [101]:
import CSP
cspProblem = CSP.CSP(lsVariables, dctVarInfo, dctDomains, dctPotNeighbors, dctConfNeighbors, None)
cspProblem.Solve()

Allowed in group: 1 
Allowed in group: 2 
Allowed in group: 3 


## Code from CSP.py 

### Class declaration

In [102]:
import collections

class CSP(object):
    def __init__(self, variables, variableInfo, domains, potNeighbors, confNeighbors, constraints):
        self.variables = variables
        self.valInfo = variableInfo
        self.domains = domains
        self.notConstrainedDomains = None
        self.potNeighbors = potNeighbors
        self.confNeighbors = confNeighbors
        self.constraints = constraints
        self.assignments = collections.defaultdict(list)

### Solve function: Takes in CSP object and performs actions needed to solve CSP problem. Once solved it returns solution to hw2cs561s2018.py file.

In [107]:
    def Solve(self):
        lsDomains = self.domains
        if len(self.assignments) == len(self.variables):
            return self.assignments
        
        currentCountry = self.GetVarMRV()
        lsReorderedDomains = DomainValLCV(currentCountry)
        
        for value in lsDomains[currentCountry]:
            if 0 == self.CheckConstraints(currentCountry, value, dctAssignments):
                print("Allowed in group: %d " % value)

### GetVar function: Takes in self object, and finds unassigned variables and returns.

In [105]:
    def GetVarMRV(self):
        for country in self.variables:
            if country not in self.assignments:
                return country

### CheckConstraints function: Takes in CSP object, currentCountry to check constraints, the value to check current variables assigned in groups, and the dictionary of assignments.

In [106]:
    def CheckConstraints(self, currentCountry, value):
        constraints = 0
        if value in dctAssignments:
            for country in dctAssignments[value]:
                if self.potNeighbors[country] == self.potNeighbors[currentCountry] or \
                    self.confNeighbors[country] == self.confNeighbors[currentCountry]:
                    constraints = constraints + 1

        elif self.potNeighbors[country] != self.potNeighbors[currentCountry] and \
            self.confNeighbors[country] != self.confNeighbors[currentCountry]:
            return 0